# 1. Aquisição de Dados

In [2]:
import pandas as pd


data = pd.read_csv('stackoverflow_perguntas.csv')

data.head(5)

,Perguntas,Tags
0,Possuo um projeto Node.js porém preciso criar ...,node.js
1,"Gostaria de fazer testes unitários no Node.js,...",node.js
2,Como inverter a ordem com que o jQuery itera u...,jquery
3,Eu tenho uma página onde pretendo utilizar um ...,html
4,Como exibir os dados retornados do FireStore e...,html angular


# 2. Pré-processamento dos Dados

In [3]:
print(data.Tags.unique())

['node.js' 'jquery' 'html' 'html angular ' 'html ' 'angular' 'angular '
 'jquery html  ' 'jquery ' 'jquery html' 'jquery html ' 'html angular'
 'angular node.js ' 'html  ' 'jquery html angular' 'node.js '
 'html jquery' 'html jquery ' 'jquery angular  ' 'html node.js' 'jquery  '
 'angular node.js' 'jquery angular' 'html node.js ' 'jquery node.js '
 'angular  ' 'jquery angular ' 'jquery html angular ' 'node.js html '
 ' node.js' 'node.js html' 'html angular  ' 'jquery node.js'
 'angular html' 'html angular  node.js' 'jquery html node.js'
 'html angular node.js']


In [4]:
labels = []

## Obtendo todas as tags individuais
for tags in data.Tags.unique():
    tags = tags.lower()
    tags = tags.strip()
    for tag in tags.split():
        if tag not in labels:
            labels.append(tag)

## Criando colunas para cada tag 
for label in labels:
    label_column = []

    for tags in data.Tags:
        if label in tags:
            label_column.append(1)
        else:
            label_column.append(0)

    data[label] = label_column

In [5]:
labels

['node.js', 'jquery', 'html', 'angular']

In [6]:
all_tags = zip(data[labels[0]], data[labels[1]], data[labels[2]], data[labels[3]])
data['all_tags'] = list(all_tags)

data.head(5)

,Perguntas,Tags,node.js,jquery,html,angular,all_tags
0,Possuo um projeto Node.js porém preciso criar ...,node.js,1,0,0,0,"(1, 0, 0, 0)"
1,"Gostaria de fazer testes unitários no Node.js,...",node.js,1,0,0,0,"(1, 0, 0, 0)"
2,Como inverter a ordem com que o jQuery itera u...,jquery,0,1,0,0,"(0, 1, 0, 0)"
3,Eu tenho uma página onde pretendo utilizar um ...,html,0,0,1,0,"(0, 0, 1, 0)"
4,Como exibir os dados retornados do FireStore e...,html angular,0,0,1,1,"(0, 0, 1, 1)"


# 3. Modelagem

In [7]:
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.model_selection import KFold, cross_validate
from sklearn.metrics import make_scorer, hamming_loss

X = np.asarray(list(data["Perguntas"]))
y = np.asarray(list(data["all_tags"]))

In [8]:
from sklearn.multiclass import OneVsRestClassifier

pipeline = Pipeline([
    ('vectorizer', TfidfVectorizer()),
    ('classifier', OneVsRestClassifier(LinearSVC(dual=False, random_state=0)))])

scores = cross_validate(pipeline, X, y, cv=KFold(n_splits=10, shuffle=True), scoring=make_scorer(hamming_loss), return_train_score=True)

mean = scores['test_score'].mean()
std = scores['test_score'].std()

print(f"Hamming Loss: {mean*100:.2f}% | {(mean + 2*std)*100:.2f}% ~ {(mean - 2*std)*100:.2f}%")

Hamming Loss: 18.54% | 19.90% ~ 17.18%


In [9]:
from skmultilearn.problem_transform import ClassifierChain

# A função custom_hamming_scorer foi criada para calcular a perda de Hamming em tarefas de classificação multirrótulo
# com o modelo ClassifierChain. Isso é necessário porque as funções de pontuação padrão do scikit-learn esperam um 
# atributo 'classes_', que não está presente no ClassifierChain. Assim, essa função personalizada realiza a previsão 
# e o cálculo da perda de Hamming de forma adequada para este tipo de modelo.

def custom_hamming_scorer(estimator, X, y):
    y_pred = estimator.predict(X)
    return hamming_loss(y, y_pred)

pipeline = Pipeline([
    ('vectorizer', TfidfVectorizer()),
    ('classifier', ClassifierChain(LinearSVC(dual=False, random_state=0)))
])

scores = cross_validate(pipeline, X, y, cv=KFold(n_splits=10, shuffle=True), scoring=custom_hamming_scorer, return_train_score=True)

mean = scores['test_score'].mean()
std = scores['test_score'].std()

print(f"Hamming Loss: {mean*100:.2f}% | {(mean + 2*std)*100:.2f}% ~ {(mean - 2*std)*100:.2f}%")


Hamming Loss: 19.55% | 21.04% ~ 18.07%


In [ ]:
from skmultilearn.adapt import MLkNN

pipeline = Pipeline([
    ('vectorizer', TfidfVectorizer()),
    ('classifier', MLkNN())
])

scores = cross_validate(pipeline, X, y, cv=KFold(n_splits=10, shuffle=True), scoring=custom_hamming_scorer, return_train_score=True)

mean = scores['test_score'].mean()
std = scores['test_score'].std()


print(f"Hamming Loss: {mean*100:.2f}% | {(mean + 2*std)*100:.2f}% ~ {(mean - 2*std)*100:.2f}%")